<a href="https://colab.research.google.com/github/lizabalyura23/-/blob/main/model%2Bbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

# Устанавливаем пути к директориям для обучающего, валидационного и тестового наборов данных
train_dir = pathlib.Path("/content/drive/MyDrive/datasets/train")
test_dir = pathlib.Path("/content/drive/MyDrive/datasets/test")
val_dir = pathlib.Path("/content/drive/MyDrive/datasets/validation")


# Устанавливаем размеры изображений
img_height = 224
img_weigth = 224

# Загружаем наборы данных с помощью image_dataset_from_directory
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir)
val_ds = tf.keras.utils.image_dataset_from_directory(val_dir)
# Получаем имена классов и количество классов
class_names = dict(zip(train_ds.class_names, range(len(train_ds.class_names))))
num_classes = len(class_names)

# Создаем генератор изображений для аугментации данных и их предобработки
train_generator = ImageDataGenerator(
preprocessing_function = mobilenet_v2.preprocess_input,
rotation_range = 32,
zoom_range = 0.2,
width_shift_range = 0.2,
height_shift_range = 0.2,
shear_range = 0.2,
horizontal_flip = True,
fill_mode = "nearest")

# Загружаем и предобрабатываем обучающие данные
train = train_generator.flow_from_directory(train_dir,
target_size = (img_height,img_weigth),
# изображение имеет 3 цветовых канала
color_mode = "rgb",
# создаем бинарные признаки меток класса
class_mode = "categorical",
batch_size = 32,
shuffle = True,
seed = 123)

# Загружаем и предобрабатываем валидационные данные
validation = train_generator.flow_from_directory(val_dir,
target_size = (img_height,img_weigth),
# изображение имеет 3 цветовых канала
color_mode = "rgb",
# создаем бинарные признаки меток класса
class_mode = "categorical",
batch_size = 32,
shuffle = True,
seed = 123)

# Загружаем модель MobileNetV2 с предварительно обученными весами ImageNet
mobilenet_ = MobileNetV2(
input_shape = (img_height,img_weigth,3),
include_top = False,
weights = 'imagenet',
pooling = 'avg')

# Замораживаем слои базовой модели
mobilenet_.trainable = False


# Создаем модель, добавив настраиваемые слои поверх базовой модели
inputs = mobilenet_.input
x = Dense(128, activation = 'relu')(mobilenet_.output)
x = Dense(128, activation = 'relu')(x)
outputs = Dense(num_classes , activation = 'softmax')(x)

# Определяем полную модель
mobilenet = Model(inputs = inputs, outputs = outputs)

# Устанавливаем раннюю остановку для предотвращения переобучения и сохранение лучших весов модели
early_stopping = EarlyStopping(
	monitor='val_loss',
	mode='min',
	patience = 2,
	verbose=1,
	restore_best_weights=True,
)
checkpoint =ModelCheckpoint('/content/drive/MyDrive/fruit224mobile.h5',
                        	monitor = 'val_loss',
                        	mode = 'min',
                       	save_best_only = True)

# Компилируем модель с оптимизатором Adam и категориальной кросс-энтропией в качестве функции потерь
callbacks = [early_stopping, checkpoint]

mobilenet.compile(optimizer=Adam(learning_rate=0.001), loss ='categorical_crossentropy',metrics = ['accuracy'])

# Загружаем и предобрабатываем тестовые данные
test = train_generator.flow_from_directory(test_dir,
target_size = (224,224),
color_mode = "rgb",
class_mode = "categorical",
batch_size = 32,
shuffle = False)

# Обучаем модель на обучающих данных и проводим валидацию на валидационных данных
history = mobilenet.fit(
train, validation_data = validation,
epochs = 20,
callbacks = callbacks)

# Оцениваем модель на тестовых данных
(eval_loss, eval_accuracy) = mobilenet.evaluate(test)



Found 3135 files belonging to 36 classes.
Found 359 files belonging to 36 classes.
Found 351 files belonging to 36 classes.
Found 3135 images belonging to 36 classes.
Found 351 images belonging to 36 classes.
9406464/9406464 [==============================] - 0s 0us/step
Found 359 images belonging to 36 classes.
Epoch 1/20
 6/98 [>.............................] - ETA: 13:26 - loss: 3.6028 - accuracy: 0.0625

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


98/98 [==============================] - ETA: 0s - loss: 1.8582 - accuracy: 0.5072 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


98/98 [==============================] - 1109s 11s/step - loss: 1.8582 - accuracy: 0.5072 - val_loss: 0.6189 - val_accuracy: 0.8405
Epoch 2/20
98/98 [==============================] - 388s 4s/step - loss: 0.8034 - accuracy: 0.7528 - val_loss: 0.4715 - val_accuracy: 0.8291
Epoch 3/20
98/98 [==============================] - 335s 3s/step - loss: 0.5902 - accuracy: 0.8045 - val_loss: 0.3808 - val_accuracy: 0.8661
Epoch 4/20
98/98 [==============================] - 352s 4s/step - loss: 0.4707 - accuracy: 0.8376 - val_loss: 0.3312 - val_accuracy: 0.8860
Epoch 5/20
98/98 [==============================] - 350s 4s/step - loss: 0.3512 - accuracy: 0.8759 - val_loss: 0.2587 - val_accuracy: 0.9117
Epoch 7/20
98/98 [==============================] - 339s 3s/step - loss: 0.3283 - accuracy: 0.8788 - val_loss: 0.2893 - val_accuracy: 0.9117
Epoch 8/20
98/98 [==============================] - 331s 3s/step - loss: 0.2825 - accuracy: 0.8976 - val_loss: 0.2694 - val_accuracy: 0.9288
Epoch 8: early stoppin

In [7]:
import cv2
import numpy as np
import tensorflow as tf

# Загрузка модели
model_path = '/content/drive/MyDrive/fruit224mobile.h5'
model = tf.keras.models.load_model(model_path)

# Маппинг классов
class_names = {
    0: 'Яблоко',
    1: 'Банан',
    2: 'Свекла',
    3: 'Болгарский перец',
    4: 'Капуста',
    5: 'Стручковый перец',
    6: 'Морковь',
    7: 'Цветная капуста',
    8: 'Перец чили',
    9: 'Кукуруза',
    10: 'Огурец',
    11: 'Баклажан',
    12: 'Чеснок',
    13: 'Имбирь',
    14: 'Виноград',
    15: 'Халапеньо',
    16: 'Киви',
    17: 'Лимон',
    18: 'Латук',
    19: 'Манго',
    20: 'Лук',
    21: 'Апельсин',
    22: 'Паприка',
    23: 'Груша',
    24: 'Горох',
    25: 'Ананас',
    26: 'Гранат',
    27: 'Картофель',
    28: 'Редька',
    29: 'Соевые бобы',
    30: 'Шпинат',
    31: 'Сладкая кукуруза',
    32: 'Батат',
    33: 'Помидор',
    34: 'Репа',
    35: 'Арбуз'
}

# Функция для предсказания класса изображения
def predict_image(image_path):
    # Загрузка и предобработка изображения
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)

    # Предсказание класса
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions[0])
    predicted_label = class_names[predicted_class]

    return predicted_label

# Тестирование функции предсказания
image_path = '/content/Image_3.jpg'  # Замените на путь к тестовому изображению
predicted_label = predict_image(image_path)
print(f"Предсказанный класс: {predicted_label}")



1/1 [==============================] - 1s 843ms/step
Предсказанный класс: Банан


In [9]:
!pip install aiogram

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 7.3 MB/s eta 0:00:00


In [10]:
#ПОБЕДА!!!!!!
import logging
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
import numpy as np
import tensorflow as tf
from aiogram import F
import asyncio
import cv2

# Настройка логирования
logging.basicConfig(level=logging.INFO)

# Создание экземпляра бота с токеном
bot = Bot(token='6959067623:AAFphw5BAC0dqooMfve_TCFnwXlz-m8eEr4')
dp = Dispatcher()

# Функция для предсказания класса
def predictions(img_path):
    classifier = tf.keras.models.load_model("/content/drive/MyDrive/fruit224mobile.h5")
    class_labels = {
        0: 'Яблоко',
        1: 'Банан',
        2: 'Свекла',
        3: 'Болгарский перец',
        4: 'Капуста',
        5: 'Стручковый перец',
        6: 'Морковь',
        7: 'Цветная капуста',
        8: 'Перец чили',
        9: 'Кукуруза',
        10: 'Огурец',
        11: 'Баклажан',
        12: 'Чеснок',
        13: 'Имбирь',
        14: 'Виноград',
        15: 'Халапеньо',
        16: 'Киви',
        17: 'Лимон',
        18: 'Латук',
        19: 'Манго',
        20: 'Лук',
        21: 'Апельсин',
        22: 'Паприка',
        23: 'Груша',
        24: 'Горох',
        25: 'Ананас',
        26: 'Гранат',
        27: 'Картофель',
        28: 'Редька',
        29: 'Соевые бобы',
        30: 'Шпинат',
        31: 'Сладкая кукуруза',
        32: 'Батат',
        33: 'Помидор',
        34: 'Репа',
        35: 'Арбуз'
    }
    # Загрузка изображения
    img = cv2.imread(img_path)
    # Изменение размера изображения
    img = cv2.resize(img, (224, 224))
    # Конвертация изображения из BGR в RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Добавление измерения для batch
    img = np.expand_dims(img, axis=0)
    # Предобработка изображения
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)


    # Предсказание класса
    predictions = classifier.predict(img)
    predicted_class = np.argmax(predictions[0])
    predicted_label = class_labels[predicted_class]

    return predicted_label

# Функция для загрузки изображения и преобразования его в массив
def get_img_array(img_path, size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

# Обработчик команды /start
@dp.message(Command("start"))
async def start(message: types.Message):
    await message.reply('Привет! Я бот, распознающий овощи и фрукты')

# Обработчик команды /help
@dp.message(Command("help"))
async def help(message: types.Message):
    await message.reply('Просто отправьте мне изображение, которое содержит овощ или фрукт')


# Обработка полученных фотографий
@dp.message(F.photo)
async def download_photo(message: types.Message, bot: Bot):
    # Сохранение изображения во временный файл
    img_path = f"tmp/{message.photo[-1].file_id}.jpg"
    await bot.download(
        message.photo[-1],
        destination=img_path
    )
    print(f"Downloaded photo path: {img_path}")
    # Предсказание класса объекта на изображении
    pred = predictions(img_path)
    await message.answer(f"Я думаю, что это {pred} 😊")


#запуск бота
async def main():
    await dp.start_polling(bot)



RuntimeError: asyncio.run() cannot be called from a running event loop

In [13]:
#победа
await main()

Downloaded photo path: tmp/AgACAgIAAxkBAAMxZk9C6AM90t05EezpZDEzY8J_8qwAAlHkMRtz7IBKzNWOM34TT6oBAAMCAAN5AAM1BA.jpg


1/1 [==============================] - 1s 856ms/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM0Zk9DGidmSW6cLkb73c5UfT6PJTQAAoLVMRt-WnFK2WsOHOYpktwBAAMCAAN4AAM1BA.jpg
1/1 [==============================] - 1s 1s/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM2Zk9DIJR6Ff4qupArI0MCx5IjZo4AAlPkMRtz7IBKpFeE4BF5_4IBAAMCAAN5AAM1BA.jpg
1/1 [==============================] - 1s 858ms/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM4Zk9D0c882A63iYtFXnWukGINWrIAAn7dMRsSOXhKyh8myYERx9kBAAMCAAN5AAM1BA.jpg
1/1 [==============================] - 1s 876ms/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM6Zk9D8Vr88UaPFYZz16WBAvaGAU4AAn_dMRsSOXhKMFfPc_-A9BcBAAMCAAN5AAM1BA.jpg
1/1 [==============================] - 1s 860ms/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM8Zk9EDMkeBmrtvPTm7EPsPFT4bAIAAoHdMRsSOXhK9wXG-t6ferABAAMCAAN5AAM1BA.jpg
1/1 [==============================] - 1s 1s/step
Downloaded photo path: tmp/AgACAgIAAxkBAAM-Zk9ERKhiK0lfE6t1iayEtVeE9pIAAoLdMRsSOXhKn_MilyZu3eQBAAMCAAN5AAM1BA.jpg
1/1 